In [1]:
import itertools
import numpy as np
from numpy import linalg as LA
import math

In [118]:
# specify interaction and hopping terms
model = [
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]}
]

model2 = [
    {"U": 2, "neighbors": [1], "hoppings": [0,1]},
    {"U": 2, "neighbors": [0], "hoppings": [1,0]}
]

model2 = [
    {"U": 2, "neighbors": [1,2,5], "hoppings": [0,1,1,0,0,1]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0]},
    {"U": 2, "neighbors": [0,1,3], "hoppings": [1,1,0,1,0,0]},
    {"U": 2, "neighbors": [2,4,5], "hoppings": [0,0,1,0,1,1]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1]},
    {"U": 2, "neighbors": [0,3,4], "hoppings": [1,0,0,1,1,0]}
]

# Gutzwiller model
t = t2 = t3 = 1
model = [
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,t,0,t,0,t2,0,0]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [t,0,t,0,0,0,t2,0]},
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,t,0,t,0,0,0,t2]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [t,0,t,0,t2,0,0,0]},
    {"U": 0, "neighbors": [3,5,7], "hoppings": [0,0,0,t2,0,t3,0,t3]},
    {"U": 0, "neighbors": [0,4,6], "hoppings": [t2,0,0,0,t3,0,t3,0]},
    {"U": 0, "neighbors": [1,5,7], "hoppings": [0,t2,0,0,0,t3,0,t3]},
    {"U": 0, "neighbors": [2,4,6], "hoppings": [0,0,t2,0,t3,0,t3,0]}
]

# 0 up; 1 up; 2 up; 3 up; ... ||  0 down; 1 down; 2 down; 3 down; ...

In [119]:
# number of sites
N = len(model)
print(N, 'sites')

def generate_basis_states(n):
    """
    Generates basis states for half filling of N sites
    """
    result = []
    int_result = []
    for bits in itertools.combinations(range(n * 2), n):
        # only choosing N bits from N * 2 bits, so half filling
        s = ['0'] * n * 2
        for bit in bits:
            # first n bits are up, second n are down
            # sites go 0 to N - 1 for each spin
            s[bit] = '1'
        
        if (''.join(s)[:N].count('1') == N / 2): # half filling
            int_result.append(int(''.join(s),2)) # integer representation of bit basis
            result.append(''.join(s))            # bit (string) basis
    # order ascending
    return (result[::-1], int_result[::-1])

(result, basis_states) = generate_basis_states(N)

8 sites


In [120]:
# for i in range(len(result)):
#     print('{0}\t\t|{1}>\t\t{2}'.format(i+1, result[i], basis_states[i]))
print(len(result))

4900


In [121]:
# setting up U term in half-filling space
h_u = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])

#          up     down
spins = [(x[:N], x[N:]) for x in result]

for i in range(len(spins)):
    # if two particles are on the same site this is recorded in h_u. Only diagonal is affected
    # i.e. ('00', '11') --> 0*1 + 0*1 = 0
    #      ('01', '01') --> 0*0 + 1*1 = 1  both particles are on side 1
    sites = []
    for j in range(N):
        if (int(spins[i][0][j]) * int(spins[i][1][j]) == 1):
            sites.append(j)
    if (len(sites) > 0):
        h_u[i][i] = sum([model[x]["U"] for x in sites])

In [122]:
# setting up t term in half-filling space
h_t = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])

def hops(state):
    def ns(s):
        """
        Finds the possible neighbor states for a state s. Spins can't flip while hopping
        """
        hops = []
        for i in range(len(s)):
            if (s[i] == '1'):
                # check all neighbors of model[i]
                for j in model[i]["neighbors"]:
                    if s[j] == '0':
                        # count the number of 1s between i and j
                        c = s[i+1:j].count('1') if i < j else s[j+1:i].count('1')
                        # convert to floats to get rid of -0 bug
                        hops.append((float(i), (-1)**c * float(j)))
        return hops

    def simulate_hop(s, hop):
        """
        Returns a new state in the bit basis after hopping
        """
        s = list(s)
        s[abs(int(hop[0]))] = '0'
        s[abs(int(hop[1]))] = '1'
        return "".join(s)

    
    neighbor_states = []
    # find possible hops for particles with up spin
    up_hops = ns(state[0])
    # find possible hops for particles with down spin
    down_hops = ns(state[1])

    start = basis_states.index(int(state[0] + state[1], 2))
    for i in range(len(down_hops)):
        hop = state[0] + simulate_hop(state[1], down_hops[i])
        hop_index = basis_states.index(int(hop, 2))
        if (math.copysign(1, down_hops[i][1]) == -1.0):
            # model[down_hops[i][0]] selects starting site
            # hops to down_hops[i][1], use model array to get specific t value for the hop
            # entry in h_t is positive because we multiply by negative t
            h_t[start][hop_index] = model[int(down_hops[i][0])]["hoppings"][abs(int(down_hops[i][1]))]
        else:
            # entry in h_t is negative because we multiply by negative t
            h_t[start][hop_index] = -model[int(down_hops[i][0])]["hoppings"][int(down_hops[i][1])]


    for i in range(len(up_hops)):
        hop = simulate_hop(state[0], up_hops[i]) + state[1]
        hop_index = basis_states.index(int(hop, 2))
        if (math.copysign(1, up_hops[i][1]) == -1.0):
            h_t[start][hop_index] = model[int(up_hops[i][0])]["hoppings"][abs(int(up_hops[i][1]))]
        else:
            h_t[start][hop_index] = -model[int(up_hops[i][0])]["hoppings"][int(up_hops[i][1])]

for i in range(len(spins)):
    hops(spins[i])


In [123]:
h = h_u + h_t
if (len(model) < 6):
    np.savetxt('h.txt', h, '%-2d', ', ')
print(h)
print(np.count_nonzero(h))
print(len(h[0]))

[[ 0 -1  0 ...  0  0  0]
 [-1  0 -1 ...  0  0  0]
 [ 0 -1  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  6 -1  0]
 [ 0  0  0 ... -1  6 -1]
 [ 0  0  0 ...  0 -1  8]]
70849
4900


In [124]:
w, v = LA.eigh(h)
print(w[0:10])

[-10.56653368  -8.92053781  -8.85403312  -8.85403312  -8.69367622
  -8.64086534  -8.63969713  -8.63969713  -8.5996555   -8.59517009]


In [79]:
simplified_h_t =  np.array([[ 0 for i in range(N)] for j in range(N)])
for i in range(N):
    for j in model[i]["neighbors"]:
        simplified_h_t[i][j] = -1
print(simplified_h_t)

[[ 0 -1  0 -1]
 [-1  0 -1  0]
 [ 0 -1  0 -1]
 [-1  0 -1  0]]


In [80]:
w2, v2 = LA.eigh(simplified_h_t)
Q = v2[:,0:N//2].T
print(Q)

[[ 5.00000000e-01  5.00000000e-01  5.00000000e-01  5.00000000e-01]
 [-7.07106781e-01 -2.45326947e-17  7.07106781e-01 -2.45326947e-17]]


In [38]:
def creation(state, site, spin):
    """
    Creates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]
    if (particle[site] == '1'):
        # Can't add a particle to an already filled site, return vacuum state
        return ''
    else:
        particle[site] = '1'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state

def annihilation(state, site, spin):
    """
    Annihilates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]

    if (particle[site] == '0'):
        # State has been destroyed, so return vacuum state
        return ''
    else:
        particle[site] = '0'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state


In [39]:
# Building the single particle density matrix
# 1 for up, 0 for down
# sigma1, sigma2 up to N. i, j sites

spdm = {}

for sigma1 in range(2):
    for i in range(N):        
        for sigma2 in range(2):
            for j in range(N):
                # setting up M(i, j, sigma1, sigma2)
                arr = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
                
                for mu in range(len(result)):
                    for nu in range(len(result)):
                        # < nu | a_{i, sigma1}^dagger a_{j, sigma2} | mu >
                
                        phi1 = annihilation(result[nu], i, sigma1)
                        phi2 = annihilation(result[mu], j, sigma2)
                        if ((phi1 != '' and phi2 != '') and phi1 == phi2):
                            arr[mu][nu] = 1
                spdm[str(i)+str(sigma1)+str(j)+str(sigma2)] = arr

In [40]:
almost_spdm_arr = np.array([0.0 for i in range(len(spdm))])
for i, j in enumerate(spdm.values()):
    almost_spdm_arr[i] = np.dot(v[:, 0].conj().T, np.dot(j, v[:, 0]))
asa = np.reshape(almost_spdm_arr, (2*N,2*N))
print(np.around(asa, decimals=2))
print(np.trace(asa))

[[ 0.5   0.23 -0.   -0.23  0.    0.    0.    0.  ]
 [ 0.23  0.5   0.23  0.    0.    0.    0.    0.  ]
 [-0.    0.23  0.5   0.23  0.    0.    0.    0.  ]
 [-0.23  0.    0.23  0.5   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.5   0.23  0.   -0.23]
 [ 0.    0.    0.    0.    0.23  0.5   0.23  0.  ]
 [ 0.    0.    0.    0.    0.    0.23  0.5   0.23]
 [ 0.    0.    0.    0.   -0.23  0.    0.23  0.5 ]]
3.999999999999999
